In [5]:
!pip install farm-haystack
!pip install torch
!pip install farm-haystack[inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.0/764.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... don

In [ ]:
# Install haystack and other required libraries
# pip install farm-haystack
# pip install faiss-cpu

from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline

# Initialize the FAISS document store
document_store=InMemoryDocumentStore()
#document_store = FAISSDocumentStore(sql_url="sqlite:///my_db.sqlite", faiss_index_factory_str="Flat")

# Index some sample documents
# Here, documents is a list of dictionaries where each dictionary represents a document
# Each document must have a unique "id" and "text" key
documents = [
    {"id": "doc1", "text": "Sample document 1"},
    {"id": "doc2", "text": "Sample document 2"},
    # Add more documents as needed
]

# Write the documents to the document store
document_store.write_documents(documents)

# Initialize the Dense Passage Retriever (DPR) for retrieval
retriever = DensePassageRetriever(document_store=document_store)

# Load a BERT-based reader for question answering
# You can also use other reader models like RoBERTa or Electra
reader = FARMReader(model_name_or_path="bert-base-uncased")

# Build the Extractive Question Answering Pipeline
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# Ask a question and get the answers
question = "What is the sample document 1 about?"
prediction = pipeline.run(query=question, top_k_retriever=5, top_k_reader=1)

# Print the answer(s)
for answer in prediction["answers"]:
    print(f"Answer: {answer['answer']} | Context: {answer['context']}")


TypeError: ignored

In [13]:

import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [3]:
#Initialize the document store
from haystack.document_stores import InMemoryDocumentStore
document_store=InMemoryDocumentStore(use_bm25=True)

In [6]:
#preparing documents
from haystack.utils import fetch_archive_from_http
doc_dir="/content/sample_data/haystack"
fetch_archive_from_http( url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip",output_dir=doc_dir)

False

In [7]:
#convert the files into document objects
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index=[doc_dir +'/'+ f for f in os.listdir(doc_dir)]
indexing_pipeline=TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(files_to_index)




Updating BM25 representation...: 100%|██████████| 2359/2359 [00:00<00:00, 7025.29 docs/s]


{'documents': [<Document: {'content': '\n\n"\'\'\'The Gift\'\'\'" is the seventh episode of the fifth season of HBO\'s fantasy television series \'\'Game of Thrones\'\', and the 47th overall. The episode was written by the series\' creators David Benioff and D. B. Weiss, and directed by Miguel Sapochnik, his directorial debut for the series.\n\nThis episode marks the final appearance for Peter Vaughan (Maester Aemon).\n\n==Plot==\n\n===At the Wall===\nJon leaves for Hardhome with Tormund and some rangers, in spite of Ser Alliser\'s complaints. Before they leave, Sam gives Jon a bag of dragonglass weapons, reminding him of their use against the White Walkers. Sam and Gilly visit Maester Aemon, who has fallen ill and dies at nightfall. In the morning, Sam eulogizes him before lighting his funeral pyre. Afterward, Gilly is attacked in the dining hall by two brothers. Sam attempts to intervene and is badly beaten before Jon\'s wolf, Ghost, scares off the brothers. Gilly tends to Sam and th

In [9]:
#initialize Retriever
from haystack.nodes import BM25Retriever

Retriever=BM25Retriever(document_store=document_store)

In [1]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


In [19]:
from haystack.pipelines import ExtractiveQAPipeline
pipe=ExtractiveQAPipeline(reader,Retriever)

In [22]:
prediction = pipe.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

#prediction = pipe.run(query="Who is the father of Arya Stark?", top_k_retriever=5, top_k_reader=1)

Inferencing Samples: 100%|██████████| 1/1 [00:22<00:00, 22.48s/ Batches]


In [23]:
print(prediction)


{'query': 'Who is the father of Arya Stark?', 'no_ans_gap': 11.405381202697754, 'answers': [<Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.993372917175293, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 207, 'end': 213}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_ids': ['9e3c863097d66aeed9992e0b6bf1f2f4'], 'meta': {'_split_id': 3}}>, <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.9753611087799072, 'context': "k in the television series.\n\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's h", 'offsets_in_document': [{'start': 630, 'end': 633}], 'offsets_in_context': [{'start': 74, 'end': 77}], 'document_ids': ['7d3360fa29130e69ea6b2ba5c5a8f9c8'], 'meta': {'_split_id': 10}}>, <Answer {'answer': 'Lord Eddard Stark', 'type': 